# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%reload_ext dotenv
%dotenv

In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\dimam\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
# Write your code below.
import os
from glob import glob

# Load the PRICE_DATA environment variable
price_data_dir = os.getenv('PRICE_DATA')
print(price_data_dir)

parquet_files = glob(os.path.join(price_data_dir, '*/*.parquet'))
parquet_files[:5]
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    Adj_Close = lambda x: x['Close']
).assign(
    Adj_Close_lag_1 = lambda x: x['Adj_Close'].shift(1)
).assign(    
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))

C:\Users\dimam\AppData\Local\Temp\ipykernel_20712\188237708.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [6]:
dd_feat.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,Adj_Close,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.995835,35.126731,34.760216,34.825665,2039962,0.0,0.0,2013,NaN,34.825665,NaN,NaN,0.366515
A,2013-12-03 00:00:00-05:00,34.635851,34.799476,34.400232,34.642395,3462706,0.0,0.0,2013,34.825665,34.642395,34.825665,-0.005262,0.399244
A,2013-12-04 00:00:00-05:00,34.583499,35.231451,34.504962,35.067825,3377288,0.0,0.0,2013,34.642395,35.067825,34.642395,0.012281,0.726489
A,2013-12-05 00:00:00-05:00,34.917291,35.211813,34.766757,35.015465,2530939,0.0,0.0,2013,35.067825,35.015465,35.067825,-0.001493,0.445056
A,2013-12-06 00:00:00-05:00,35.185633,35.951389,35.185633,35.885941,4268513,0.0,0.0,2013,35.015465,35.885941,35.015465,0.024860,0.765756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165619,128.937943,128.165619,0.006026,1.752578
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,127.819061,128.937943,-0.008678,2.000117
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,129.264694,127.819061,0.011310,2.237759


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [9]:
# Convert to pandas DataFrame
pd_feat = dd_feat.compute()

# Add rolling average return calculation with a window of 10 days
pd_feat['rolling_avg_return'] = pd_feat['returns'].rolling(window=10).mean()

# Display the result
print(pd_feat.tail())

c:\Users\dimam\miniconda3\envs\dsi_participant\lib\site-packages\pyarrow\pandas_compat.py:766: DeprecationWarning: DatetimeTZBlock is deprecated and will be removed in a future version. Use public APIs instead.
  klass=_int.DatetimeTZBlock,


                            Date        Open        High         Low  \
ticker                                                                 
YUM    2024-01-25 00:00:00-05:00  128.551783  129.027053  127.274475   
YUM    2024-01-26 00:00:00-05:00  128.640891  129.403315  127.403198   
YUM    2024-01-29 00:00:00-05:00  127.828973  129.264694  127.026935   
YUM    2024-01-30 00:00:00-05:00  129.076565  129.928098  128.304245   
YUM    2024-01-31 00:00:00-05:00  130.155828  130.680609  127.987387   

             Close   Volume  Dividends  Stock Splits  year  Close_lag_1  \
ticker                                                                    
YUM     128.937943  1871700        0.0           0.0  2024   128.165619   
YUM     127.819061  1157000        0.0           0.0  2024   128.937943   
YUM     129.264694  1482100        0.0           0.0  2024   127.819061   
YUM     129.334000  1421500        0.0           0.0  2024   129.264694   
YUM     128.215134  2154200        0.0       

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
    - No, it is not necessary to convert to a pandas DataFrame to calculate the moving average return. 
        - Dask can handle rolling operations directly on Dask DataFrames. 
        - Here’s the code to calculate the moving average return using Dask, without converting to a pandas DataFrame:
            - dd_feat['rolling_avg_return'] = dd_feat['returns'].rolling(window=10).mean()

+ Would it have been better to do it in Dask? Why?
    - Using Dask for rolling operations is generally better due to its ability to handle large datasets efficiently, its parallel processing capabilities, and its API consistency with pandas. This approach ensures that you can work with data that exceeds your system's memory limitations while leveraging the full potential of your hardware resources.








(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.